In [86]:
#load model

import tensorflow as tf
import numpy as np
import itertools
from transformers import TFAutoModelForMaskedLM, AutoTokenizer

model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=False, output_attentions=False)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [95]:
def calculate_sentence_score(sentence_tokens,k=1000,epsilon=0.000001):
    sentence_score = 1000000000000
    sentence = ' '.join(sentence_tokens)
    # inputs = tokenizer(sentence, return_tensors="tf")
    # input_ids = inputs["input_ids"]
    for i,token in enumerate(sentence_tokens):
    # for i in range(len(input_ids)):
        found = False
        # token =  tokenizer.convert_ids_to_tokens([input_ids[i]])
        # token = sentence_tokens[i]
        # masked_input_ids = tf.tensor_scatter_nd_update(input_ids, [[0, i]], [tokenizer.mask_token_id])
        masked_text = ' '.join(sentence_tokens[:i])+' [MASK] '+' '.join(sentence_tokens[i+1:])

        
        input_ids = tokenizer.encode(masked_text, return_tensors="tf")

        outputs = model(input_ids)
        print(outputs[-1])
        logits = outputs.logits
        # logits = model.__call__(input_ids)[0]
        mask_token_index = tf.where(input_ids == tokenizer.mask_token_id)[0, 1]
        mask_token_logits = logits[0, mask_token_index, :]
        top_k_values, top_k_indices = tf.math.top_k(mask_token_logits, k=k)
        predicted_token_indices = top_k_indices.numpy()
        probabilities = tf.nn.softmax(top_k_values).numpy()
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_indices)
        for j,predicted_token in enumerate(predicted_tokens):
            if predicted_token == token:
                sentence_score *= probabilities[j]
                found = True
                break
        if not found:
            sentence_score *= epsilon
        # print(token)
        # for i in range(k):
        #     print(predicted_tokens[i],probabilities[i])
        # print('----')
    return sentence_score

In [88]:
def clean_permutations(permutations):
    return permutations

In [89]:
import time
def find_ordered_sentence(sentence):
    tokens = tokenizer.tokenize(sentence)
    permutations = list(itertools.permutations(tokens))
    cleaned_permutations = clean_permutations(permutations)
    scores = []
    for i,permutation in enumerate(cleaned_permutations):
        permutation_sentence = ' '.join(permutation)
        score = calculate_sentence_score(list(permutation))
        print(permutation_sentence)
        print(score)
        scores.append((permutation_sentence,score))
        print(i+1,'/',len(permutations))
    scores.sort(key=lambda x: x[1], reverse=True)
    for score in scores:
        print(score)
    return scores[0][1]

In [96]:
#main code
find_ordered_sentence('شد خانه خراب')

tf.Tensor(
[[[-8.249292  -7.8504586 -4.190132  ... -6.3265657 -5.8598886 -5.0887065]
  [-7.5560904 -7.310338  -5.4801974 ... -7.349765  -7.0495486 -4.5247097]
  [-8.363945  -8.286933  -7.5320415 ... -8.101752  -6.7629743 -6.6129885]
  [-7.0748463 -6.1730356 -5.0125318 ... -7.770436  -1.3121433 -3.093759 ]
  [-5.3221436 -5.760626  -7.0017366 ... -4.122114  -2.3114796 -2.572641 ]]], shape=(1, 5, 100000), dtype=float32)
tf.Tensor(
[[[ -9.703397   -9.22757    -6.2324033 ...  -7.6335425  -8.297429
    -7.1577544]
  [-10.257478   -9.560696   -7.015505  ...  -9.4152355  -9.79613
    -6.700428 ]
  [ -6.2258716  -4.940087   -8.36199   ...  -4.187895   -4.8127046
    -3.8332863]
  [ -9.152376   -8.387696   -7.1129246 ...  -8.585529   -6.101666
    -6.8454194]
  [ -6.6361995  -7.9997687  -7.9749117 ...  -6.400436   -5.4669805
    -4.5484853]]], shape=(1, 5, 100000), dtype=float32)
tf.Tensor(
[[[-9.148305  -8.682382  -4.4242907 ... -7.0923715 -7.090711  -7.1360064]
  [-8.426776  -8.802432  -6.6316

2104356.07099184

In [7]:
#delete this cell

def calculate_sentence_score(sentence_tokens):
    sentence_score = 1
    sentence = ' '.join(sentence_tokens)
    for i,token in enumerate(sentence_tokens):
        found = False
        masked_text = ' '.join(tokens[:i])+' [MASK] '+' '.join(tokens[i+1:])

        inputs = tokenizer(masked_text, return_tensors="tf")
        outputs = model(inputs)
        logits = outputs.logits

        mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
        # mask_token_index = i
        mask_token_logits = logits[0, mask_token_index, :]

        k = 1000

        top_k_values, top_k_indices = tf.math.top_k(mask_token_logits, k=k)
        predicted_token_indices = top_k_indices.numpy()
        probabilities = tf.nn.softmax(top_k_values).numpy()
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_indices)

        for j,predicted_token in enumerate(predicted_tokens):
            if predicted_token == token:
                sentence_score *= probabilities[j]
                # print('found',sentence_score)
                found = True
                break

        epsilon = 0.00001

        if not found:
            sentence_score *= epsilon
            # print('not found',sentence_score)

        # print(token)
        # for i in range(k):
        #     print(predicted_tokens[i],probabilities[i])
        # print('----')

    soft_parameter = 1000000000000

    return sentence_score*soft_parameter


import itertools
text = "وقتی من و دوستم برمیگشتیم از مدرسه "
tokens = tokenizer.tokenize(text)
permutations = list(itertools.permutations(tokens))
scores = []
for i,permutation in enumerate(permutations):
    print(' '.join(permutation))
    scores.append((' '.join(permutation),calculate_sentence_score(list(permutation))))
    print(i+1,'/',len(permutations))
scores.sort(key=lambda x: x[1], reverse=True)
for score in scores:
    print(score)

وقتی من و دوستم برمیگشتیم از مدرسه
1 / 5040
وقتی من و دوستم برمیگشتیم مدرسه از
2 / 5040
وقتی من و دوستم از برمیگشتیم مدرسه
3 / 5040
وقتی من و دوستم از مدرسه برمیگشتیم
4 / 5040
وقتی من و دوستم مدرسه برمیگشتیم از
5 / 5040
وقتی من و دوستم مدرسه از برمیگشتیم
6 / 5040
وقتی من و برمیگشتیم دوستم از مدرسه
7 / 5040
وقتی من و برمیگشتیم دوستم مدرسه از
8 / 5040
وقتی من و برمیگشتیم از دوستم مدرسه
9 / 5040
وقتی من و برمیگشتیم از مدرسه دوستم
10 / 5040
وقتی من و برمیگشتیم مدرسه دوستم از
11 / 5040
وقتی من و برمیگشتیم مدرسه از دوستم
12 / 5040
وقتی من و از دوستم برمیگشتیم مدرسه
13 / 5040
وقتی من و از دوستم مدرسه برمیگشتیم
14 / 5040
وقتی من و از برمیگشتیم دوستم مدرسه
15 / 5040
وقتی من و از برمیگشتیم مدرسه دوستم
16 / 5040
وقتی من و از مدرسه دوستم برمیگشتیم
17 / 5040
وقتی من و از مدرسه برمیگشتیم دوستم
18 / 5040
وقتی من و مدرسه دوستم برمیگشتیم از
19 / 5040
وقتی من و مدرسه دوستم از برمیگشتیم
20 / 5040
وقتی من و مدرسه برمیگشتیم دوستم از
21 / 5040
وقتی من و مدرسه برمیگشتیم از دوستم
22 / 5040
وقتی من و مدرسه از 

KeyboardInterrupt: 